# 使用 function-calling 让非结构化数据自动创建订单

* 先看一下 OpenAI API 的 [function-calling-openai](./function-calling-openai-example.ipynb) 示例
* 基于 OpenAI API 实现自动创建订单 [openai-auto-create-order](./function-calling-openai-auto-create-order.ipynb)
* 基于 langchain 实现自动创建订单 [langchain-auto-create-order](./function-calling-langchain-auto-create-order.ipynb)

## 背景
客户为了快速下单，会把订单信息先录入 Excel 文档，然后通过 Excel 文档导入的方式批量创建订单。
但是不同客户可能有自己的订单格式，比如收件人姓名、收件人电话等，这些信息在客户一方的列名、顺序可能不一样。
但为了下单，需要把信息转换为我们的订单格式。
以前几乎都是通过人工的方式来适配我们的订单格式。

## 解决方案
这是一个典型的数据转换问题，可以通过命名实体识别（NER）来解决。
流程是：
1. 通过命名实体识别抽取数据中特定的参数
2. 用命名实体识别抽取的参数创建订单

在以前，要进行 NER，需要定制开发工具，或者通过人工标注数据，然后训练模型。

现在可以通过 LLM(OpenAI API) 来解决，只需要提供示例与数据，OpenAI API 就可以自动完成 NER。

并且，在 2023-06-13 发布的 [gpt-3.5-turbo-16k-0613](https://beta.openai.com/models/gpt3/versions/3.5-turbo-16k-0613) 模型中，OpenAI API 支持 function-calling，并且能自动提取非结构化数据，然后大模型可以自行分析应该调用哪个 function。

这样就能做到载入客户数据（Excel、CSV、JSON、XML、自然语言等），然后通过 function-calling 自动调用软件已有的功能，完成订单创建。

## 进一步思考
从现在的测试来看，微调过后的新版本，能自动做 NER 然后自动选择要调用的 function。

这里核心解决了两个问题：
1. 信息识别与转换
2. 信息链接

这里有很大的想象空间

In [11]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [12]:
from langchain.document_loaders.csv_loader import CSVLoader

file_path = "/Users/chenjun/Downloads/test-order.csv"

loader = CSVLoader(file_path)
data = loader.load()
print(data)
# 把 data 数组中的所有文档的 page_content 抽取出来
texts = [doc.page_content for doc in data]

[Document(page_content='客户订单号: GO202001013\n线路标识: \n收货地址: 火炬动力港\n收件人姓名: 运荔枝\n收件人电话号码: 12345678900\n货品: 肉制品\n数量（件数）: 1\n总重量（KG): \n总体积（m³）: \n总货值（元）: \n温层: 冷藏\n车型: \n取货地址: \n取货人姓名: \n取货人电话号码: \n提货时间: 2023/5/6 16:00\n送货时间: 2023/5/6 23:00\n备注: ', metadata={'source': '/Users/chenjun/Downloads/test-order.csv', 'row': 0}), Document(page_content='客户订单号: GO202001013\n线路标识: \n收货地址: 火炬动力港\n收件人姓名: 鲜生活\n收件人电话号码: 13409871234\n货品: 肉制品\n数量（件数）: 1\n总重量（KG): \n总体积（m³）: \n总货值（元）: \n温层: 冷藏\n车型: \n取货地址: \n取货人姓名: \n取货人电话号码: \n提货时间: 2023/5/6 16:00\n送货时间: 2023/5/6 23:00\n备注: ', metadata={'source': '/Users/chenjun/Downloads/test-order.csv', 'row': 1})]


In [13]:
# texts = ['给刘德华（电话13400000000）发一件吧',
#          '收货人：张学友，电话：13500000000',
#          '\{"姓名":"黎明", "手机号":"13600000000"\}',
#          '住在灯球下的郭富城，给他送一斤粉条，电话是13700000000，地址是，地址是灯球下']
texts = '。'.join(texts) + '。给刘德华（电话13400000000）送一斤吧。还有个叫张学友，电话13500000000也送。货多，不怕。哎，有个黎明，手机13600000000的是不是也应该送一斤啊？两斤吧。最后，住在灯球下的郭富城也送一单，电话是13700000000。好了，下单吧。'
print(texts)

客户订单号: GO202001013
线路标识: 
收货地址: 火炬动力港
收件人姓名: 运荔枝
收件人电话号码: 12345678900
货品: 肉制品
数量（件数）: 1
总重量（KG): 
总体积（m³）: 
总货值（元）: 
温层: 冷藏
车型: 
取货地址: 
取货人姓名: 
取货人电话号码: 
提货时间: 2023/5/6 16:00
送货时间: 2023/5/6 23:00
备注: 。客户订单号: GO202001013
线路标识: 
收货地址: 火炬动力港
收件人姓名: 鲜生活
收件人电话号码: 13409871234
货品: 肉制品
数量（件数）: 1
总重量（KG): 
总体积（m³）: 
总货值（元）: 
温层: 冷藏
车型: 
取货地址: 
取货人姓名: 
取货人电话号码: 
提货时间: 2023/5/6 16:00
送货时间: 2023/5/6 23:00
备注: 。给刘德华（电话13400000000）送一斤吧。还有个叫张学友，电话13500000000也送。货多，不怕。哎，有个黎明，手机13600000000的是不是也应该送一斤啊？两斤吧。最后，住在灯球下的郭富城也送一单，电话是13700000000。好了，下单吧。


In [14]:
import os
from dotenv import load_dotenv, find_dotenv
from MultiChatOpenAI import MultiChatOpenAI

load_dotenv(find_dotenv()) # read local .env file
# 导入 OpenAI API_KEY
# openai.api_key = os.environ['OPENAI_API_KEY']

model = 'gpt-3.5-turbo-16k-0613'

key1 = os.environ['OPENAI_API_KEY']
key2 = os.environ['OPENAI_API_KEY2']

llm = MultiChatOpenAI(keys=[key1, key2], model=model, temperature=0, verbose=True)

In [15]:
from langchain import PromptTemplate
from langchain.tools import tool


tools = [
]

In [16]:
# 尝试主动做 NER
# @tool("do_ner")
# def do_ner(ner_input):
#     """通过命名实体识别抽取数据中特定格式实体信息"""
#     ner_output_template = """ [
#         {
#             "receiverName": "张三",
#             "receiverMobile": "13800000000"
#         },
#         {
#             "receiverName": "李四",
#             "receiverMobile": "13800000001"
#         }
#      ]"""
#
#     template = """
#     请严格按照抽取格式返回数据，不得抽取多的实体信息，如果指定的实体信息缺失，可以设置空值。
#     抽取格式：
#     {ner_template}
#     数据：
#     {texts}
#     """
#
#     prompt = PromptTemplate.from_template(template)
#     format_str = prompt.format(ner_template=ner_output_template, texts=ner_input)
#
#     messages = [
#         SystemMessage(
#             content="你是一个命名实体识别模型。我会给你抽取格式与数据，请根据示例格式抽取数据中的信息返回。"
#         ),
#         HumanMessage(
#             content=format_str
#         ),
#     ]
#     results = llm.predict_messages(messages)
#     content = results.content
#     # 把字符串 content 转换为 JSON 对象
#     content = json.loads(content)
#     return content
# tools.append(do_ner)

In [17]:
@tool("create_order")
def create_order(orders):
    """用给定的参数创建订单 参数为 [{'receiverName': '', 'receiverMobile': ''}]"""
    print(orders)
    for order_info in orders:
        receiver_name = order_info["receiverName"]
        receiver_mobile = order_info["receiverMobile"]
        print(f'create order for  {receiver_name}/{receiver_mobile}\n')
    return "订单创建成功!!!!!!!!"

tools.append(create_order)
print(create_order.args_schema.schema())

{'title': 'create_orderSchemaSchema', 'type': 'object', 'properties': {'orders': {'title': 'Orders'}}, 'required': ['orders']}


In [18]:
# 创建 tool 的另一种方式
# tools = [
#     Tool(
#         name = "do_ner",
#         func=do_ner,
#         description="通过命名实体识别抽取数据中特定的参数"
#     ),
#     Tool(
#         name="create_order",
#         func=create_order,
#         description="用给定的参数创建订单"
#     )
# ]
#
# functions = [format_tool_to_openai_function(t) for t in tools]
# print(functions)

In [19]:
question = """
请根据给出的信息抽取相应的参数创建订单
1. 通过命名实体识别先对输入的信息抽取特定的参数。抽取的数据结构如下：
[{{"receiverName": "", "receiverMobile": ""}}]
2. 请用命名实体识别抽取的数组参数，不做任何修改使用创建订单接口创建订单

输入信息：
'''
{texts}
'''
"""
question_prompt = PromptTemplate.from_template(question)
question_format_str = question_prompt.format(texts=texts)
print(question_format_str)


请根据给出的信息抽取相应的参数创建订单
1. 通过命名实体识别先对输入的信息抽取特定的参数。抽取的数据结构如下：
[{"receiverName": "", "receiverMobile": ""}]
2. 请用命名实体识别抽取的数组参数，不做任何修改使用创建订单接口创建订单

输入信息：
'''
客户订单号: GO202001013
线路标识: 
收货地址: 火炬动力港
收件人姓名: 运荔枝
收件人电话号码: 12345678900
货品: 肉制品
数量（件数）: 1
总重量（KG): 
总体积（m³）: 
总货值（元）: 
温层: 冷藏
车型: 
取货地址: 
取货人姓名: 
取货人电话号码: 
提货时间: 2023/5/6 16:00
送货时间: 2023/5/6 23:00
备注: 。客户订单号: GO202001013
线路标识: 
收货地址: 火炬动力港
收件人姓名: 鲜生活
收件人电话号码: 13409871234
货品: 肉制品
数量（件数）: 1
总重量（KG): 
总体积（m³）: 
总货值（元）: 
温层: 冷藏
车型: 
取货地址: 
取货人姓名: 
取货人电话号码: 
提货时间: 2023/5/6 16:00
送货时间: 2023/5/6 23:00
备注: 。给刘德华（电话13400000000）送一斤吧。还有个叫张学友，电话13500000000也送。货多，不怕。哎，有个黎明，手机13600000000的是不是也应该送一斤啊？两斤吧。最后，住在灯球下的郭富城也送一单，电话是13700000000。好了，下单吧。
'''



In [20]:
# 直接使用 OpenAI
# message = llm.predict_messages(
#     [HumanMessage(content=question_format_str)], functions=functions
# )
# print(message)

In [21]:
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_MULTI_FUNCTIONS, verbose=True)

In [22]:
agent.run(question_format_str)



> Entering new  chain...

Invoking: `create_order` with `{'orders': [{'receiverName': '运荔枝', 'receiverMobile': '12345678900'}, {'receiverName': '鲜生活', 'receiverMobile': '13409871234'}, {'receiverName': '刘德华', 'receiverMobile': '13400000000'}, {'receiverName': '张学友', 'receiverMobile': '13500000000'}, {'receiverName': '黎明', 'receiverMobile': '13600000000'}, {'receiverName': '郭富城', 'receiverMobile': '13700000000'}]}`


[{'receiverName': '运荔枝', 'receiverMobile': '12345678900'}, {'receiverName': '鲜生活', 'receiverMobile': '13409871234'}, {'receiverName': '刘德华', 'receiverMobile': '13400000000'}, {'receiverName': '张学友', 'receiverMobile': '13500000000'}, {'receiverName': '黎明', 'receiverMobile': '13600000000'}, {'receiverName': '郭富城', 'receiverMobile': '13700000000'}]
create order for  运荔枝/12345678900

create order for  鲜生活/13409871234

create order for  刘德华/13400000000

create order for  张学友/13500000000

create order for  黎明/13600000000

create order for  郭富城/13700000000

订单创建成功!!!!!!!!订单已成功创建

'订单已成功创建！'